# 1 Binomial Logistic Regression

## 1.1 Single-Variable Logistic Regression

Suppose we are interested in the survival of passengers on the Titanic. In particular, suppose we want to know whether a passenger's gender impacted whether or not they survived. We can investigagte this using logistic regression.

In [ ]:
library(ggplot2)
library(broom)
library(tidyverse)

titanic <- read_csv("titanic_subset.csv")

In [ ]:
fm1 <- glm(survived ~ sex, titanic, family = binomial(link = "logit"))

In [ ]:
tidy(fm1)

In [ ]:
glance(fm1)

In [ ]:
# Define a function to compute the inverse logit. 
# Recall this is our link function.

invlogit <- function(eta) {
    res <- 1/(1 + exp(-eta))
    return(res)
}

In [ ]:
beta0 <- tidy(fm1)$estimate[1]
beta1 <- tidy(fm1)$estimate[2]

invlogit(beta0 + beta1 * 0)        # estimated Pr(survived | female)
invlogit(beta0 + beta1 * 1)        # estimated Pr(survived | male)

*** Note that the `fitted()` function can be used to obtain predicted probabilities instead of using `invlogit()`

---

---

## 1.2 Why do we Need Logistic Regression?

In [ ]:
# Simulate some data
n <- 500
beta0 <- 1.2
beta1 <- 0.7

x <- rnorm(n)           

pr <- invlogit(beta0 + beta1*x)

y <- rbinom(n, 1, pr)


dat <- data.frame(y, x)

# Compare linear regression to logistic; find that 
# logistic does better job of recovering betas
tidy(lm(y ~ x, dat))
tidy(glm(y ~ x, dat, family = binomial(link = "logit")))


---

---

## 1.3 Multivariate Binomial Logistic Regression

Suppose now we want to investigate the effect of both sex and age on survival. We use the model below. 

In [ ]:
fm2 <- glm(survived ~ sex + age, titanic, family = binomial(link = "logit"))

In [ ]:
tidy(fm2)

---

---

## 1.4 Logistic Regression with Interaction

In [ ]:
fm3 <- glm(survived ~ sex + age + age*sex, titanic, family = binomial(link = "logit"))

tidy(fm3)

In [ ]:
ggplot(titanic, aes(x = age, y = as.numeric(survived), color = sex)) +
    stat_smooth(method = "glm", alpha = 0.2, size = 2, aes(fill = sex)) +
    geom_point(position=position_jitter(height = 0.03, width = 0)) +
    xlab("Age") + 
    ylab("Pr(survived)")

In [ ]:
lr(fm1, fm2)

---

---

## 1.5 Predictions using Fitted Model

In [ ]:
fm2 <- glm(survived ~ age, titanic, family = binomial(link = "logit"))

predict(fm2, newdata=data.frame(age = 90), type = "response")

---

---

# 2 Poisson Regression

## 2.1 Poisson Model for Number of Procedures

Suppose we want to model the number of procedure for diabetes patients admitted to the hospital. We use several Poisson models below.

In [ ]:
library(tidyverse)

dia <- read_csv("diabetes_data_clean.csv")

In [ ]:
spec(dia)

In [ ]:
fm5 <- glm(num_procedures ~ number_diagnoses, dia, family = poisson(link = "log"))

In [ ]:
tidy(fm5)

In [ ]:
glance(fm5)

In [ ]:
dia_subset <- sample_frac(dia, 0.2)

ggplot(dia_subset, aes(x = number_diagnoses, y = num_procedures)) +
    geom_jitter(alpha = 0.3)

In [ ]:
fm6 <- glm(num_procedures ~ number_diagnoses + num_medications + number_diagnoses*num_medications, dia, family = poisson(link = "log"))

In [ ]:
tidy(fm6)

In [ ]:
ggplot(dia_subset, aes(x = number_diagnoses, y = num_procedures, colour = num_medications)) +
    geom_jitter(alpha = 0.5) + 
    stat_smooth(method = "glm")